In [17]:
import ee
import geopandas as gpd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional
import urllib.request
from calendar import monthrange
import requests
from geobr import read_municipality
import math
import time


In [18]:
ee.Initialize(project="cropyieldprediction-476612")

In [19]:
class GEEDownloader:
    """Downloader for Sentinel-2 imagery with MapBiomas crop masking from Google Earth Engine."""
    
    def __init__(self, output_dir: str = "files/gee_images"):
        """
        Initialize the GEE downloader.
        
        Parameters:
        -----------
        output_dir : str, default "files/gee_images"
            Directory where downloaded images will be saved.
        """
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.mapbiomas_asset = "projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_integration_v2"
        self.sentinel2_collection = "COPERNICUS/S2_SR_HARMONIZED"
    
    def download_municipality(
        self,
        shapefile_path: str,
        crop_type: int = 39,
        resolution: int = 10,
        tile_size: int = 1000,
        start_date: str = "2023-01-01",
        end_date: str = "2023-12-31",
        cloud_threshold: float = 30.0,
        composite_method: str = "median",
        bands: List[str] = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
        municipality_code: Optional[str] = None,
        municipality_name: Optional[str] = None,
        force_redownload: bool = False
    ) -> Path:
        """
        Download Sentinel-2 imagery for a municipality with MapBiomas crop masking.
        
        Downloads monthly composite images divided into fixed-size tiles. Only tiles
        containing the specified crop type are downloaded. Images are masked to show
        only pixels classified as the target crop in MapBiomas.
        
        Parameters:
        -----------
        shapefile_path : str
            Path to the municipality shapefile (.shp file).
        crop_type : int, default 39
            MapBiomas classification code for the crop type (39 = soybean).
        resolution : int, default 10
            Pixel resolution in meters for the downloaded images.
        tile_size : int, default 1000
            Size of each tile in meters (e.g., 1000 = 1km x 1km tiles).
            Note: In practice, larger values (e.g., 5000) are often used for efficiency.
        start_date : str, default "2023-01-01"
            Start date for the download period (YYYY-MM-DD format).
        end_date : str, default "2023-12-31"
            End date for the download period (YYYY-MM-DD format).
        cloud_threshold : float, default 30.0
            Maximum cloud cover percentage for images to be included in composite.
        composite_method : str, default "median"
            Method to composite multiple images per month: "median", "mean", or "first".
        bands : List[str], default ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"]
            List of Sentinel-2 bands to download (excluding B1, B9, B10).
        municipality_code : Optional[str], default None
            IBGE municipality code. If None, extracted from shapefile filename.
        municipality_name : Optional[str], default None
            Municipality name. If None, uses municipality_code.
        force_redownload : bool, default False
            If True, re-downloads even if files already exist. If False, skips
            municipalities that have already been downloaded for the requested date range.
        
        Returns:
        --------
        Path
            Path to the directory containing downloaded GeoTIFF files.
            Files are named: {municipality_code}_{year}-{month}_tile_{i:02d}_{j:02d}.tif
        """
        
        gdf = gpd.read_file(shapefile_path)
        geometry = gdf.geometry.values[0]
        geo_json = geometry.__geo_interface__
        ee_geometry = ee.Geometry(geo_json)
        
        if not municipality_code:
            municipality_code = Path(shapefile_path).stem
        if not municipality_name:
            municipality_name = municipality_code
        
        safe_name = "".join(c for c in municipality_name if c.isalnum() or c in (' ', '-', '_')).strip()
        mun_output_dir = self.output_dir / f"{municipality_code}_{safe_name}"
        
        # Check if municipality has already been downloaded (unless force_redownload is True)
        if not force_redownload and mun_output_dir.exists():
            existing_files = list(mun_output_dir.glob("*.tif"))
            if existing_files:
                print(f"Skipping {municipality_code} ({municipality_name}): folder exists with {len(existing_files)} files")
                return mun_output_dir
        
        # Start timer for this municipality
        start_time = time.time()
        start_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"\n[{start_timestamp}] Starting processing: {municipality_code} ({municipality_name})")
        
        mun_output_dir.mkdir(parents=True, exist_ok=True)
        
        try:
            bounds = ee_geometry.bounds().getInfo()['coordinates'][0]
        except Exception as e:
            gdf_bounds = gdf.total_bounds
            bounds = [
                [gdf_bounds[0], gdf_bounds[1]],  # min_lon, min_lat
                [gdf_bounds[2], gdf_bounds[1]],  # max_lon, min_lat
                [gdf_bounds[2], gdf_bounds[3]],  # max_lon, max_lat
                [gdf_bounds[0], gdf_bounds[3]]   # min_lon, max_lat
            ]
        min_lon, min_lat = bounds[0]
        max_lon, max_lat = bounds[2]
        
        center_lat = (min_lat + max_lat) / 2
        
        meters_per_degree_lat = 111000
        meters_per_degree_lon = 111000 * math.cos(math.radians(center_lat))
        
        tile_size_deg_lat = tile_size / meters_per_degree_lat
        tile_size_deg_lon = tile_size / meters_per_degree_lon
        
        num_tiles_lon = int((max_lon - min_lon) / tile_size_deg_lon) + 1
        num_tiles_lat = int((max_lat - min_lat) / tile_size_deg_lat) + 1
        
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        
        current = start
        while current <= end:
            year = current.year
            month = current.month
            month_end = datetime(year, month, monthrange(year, month)[1])
            if month_end > end:
                month_end = end
            
            month_start_str = current.strftime("%Y-%m-%d")
            month_end_str = month_end.strftime("%Y-%m-%d")
            
            sentinel2 = (
                ee.ImageCollection(self.sentinel2_collection)
                .filterBounds(ee_geometry)
                .filterDate(month_start_str, month_end_str)
                .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud_threshold))
            )
            
            try:
                if sentinel2.size().getInfo() == 0:
                    current = month_end + timedelta(days=1)
                    continue
            except Exception as e:
                current = month_end + timedelta(days=1)
                continue
            
            if composite_method == "median":
                composite = sentinel2.median()
            elif composite_method == "mean":
                composite = sentinel2.mean()
            elif composite_method == "first":
                composite = sentinel2.first()
            else:
                composite = sentinel2.median()
            
            rgb = composite.select(bands).clip(ee_geometry)
            
            mapbiomas = ee.Image(self.mapbiomas_asset)
            classification = mapbiomas.select(f"classification_{year}").clip(ee_geometry)
            crop_mask = classification.eq(crop_type)
            
            rgb_masked = rgb.multiply(crop_mask).updateMask(crop_mask)
            
            for i in range(num_tiles_lon):
                for j in range(num_tiles_lat):
                    tile_lon_min = min_lon + (i * tile_size_deg_lon)
                    tile_lon_max = min_lon + ((i + 1) * tile_size_deg_lon)
                    tile_lat_min = min_lat + (j * tile_size_deg_lat)
                    tile_lat_max = min_lat + ((j + 1) * tile_size_deg_lat)
                    
                    tile_geometry = ee.Geometry.Rectangle(
                        [tile_lon_min, tile_lat_min, tile_lon_max, tile_lat_max]
                    )
                    tile_geometry = tile_geometry.intersection(ee_geometry, ee.ErrorMargin(1))
                    
                    try:
                        tile_area = tile_geometry.area().getInfo()
                        if tile_area < 1e-6:
                            continue
                    except Exception as e:
                        continue
                    
                    tile_crop_mask = crop_mask.clip(tile_geometry)
                    has_crop = tile_crop_mask.reduceRegion(
                        reducer=ee.Reducer.sum(),
                        geometry=tile_geometry,
                        scale=resolution,  
                        maxPixels=1e9
                    ).getInfo()
                    
                    if not has_crop:
                        continue
                    crop_sum = sum(v for v in has_crop.values() if v is not None)
                    if crop_sum == 0:
                        continue
                    
                    tile_rgb_masked = rgb_masked.clip(tile_geometry)
                    
                    try:
                        url = tile_rgb_masked.getDownloadUrl({
                            "region": tile_geometry,
                            "scale": resolution,
                            "crs": "EPSG:4326",
                            "format": "GEO_TIFF"
                        })
                        
                        filename = f"{municipality_code}_{year}-{month:02d}_tile_{i:02d}_{j:02d}.tif"
                        filepath = mun_output_dir / filename
                        
                        urllib.request.urlretrieve(url, filepath)
                        
                        file_size = filepath.stat().st_size
                        if file_size < 1000:
                            filepath.unlink()
                            continue
                        
                    except Exception as e:
                        print(e)
                        continue
            
            current = month_end + timedelta(days=1)
        
        # Print completion message with elapsed time
        elapsed_time = time.time() - start_time
        elapsed_minutes = int(elapsed_time // 60)
        elapsed_seconds = int(elapsed_time % 60)
        end_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        num_files = len(list(mun_output_dir.glob("*.tif")))
        print(f"[{end_timestamp}] Completed: {municipality_code} ({municipality_name}) - {num_files} files in {elapsed_minutes}m {elapsed_seconds}s")
        
        return mun_output_dir


In [ ]:
shapefile_dir = Path("files/municipal_shapefiles")
shapefiles = list(shapefile_dir.glob("**/*.shp"))

for shp_file in shapefiles:
    print(f"Using shapefile: {shp_file}")
    
    gdf = gpd.read_file(shp_file)
    mun_code = shp_file.stem
    mun_name = shp_file.parent.name.split('_', 1)[1] if '_' in shp_file.parent.name else mun_code
    
    downloader = GEEDownloader()
    output_path = downloader.download_municipality(
        shapefile_path=str(shp_file),
        crop_type=39,
        resolution=10,
        tile_size=5000,
        start_date="2023-01-01",
        end_date="2023-06-30",
        cloud_threshold=30.0,
        composite_method="median",
        bands=["B2", "B3", "B4"], #, "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
        municipality_code=mun_code,
        municipality_name=mun_name
    )
    
    print(f"Download complete! Files saved to: {output_path}")
else:
    print("No shapefiles found in files/municipal_shapefiles/")


Using shapefile: files\municipal_shapefiles\4100103_Abatiá\4100103.shp
Skipping 4100103 (Abatiá): folder exists with 95 files
Download complete! Files saved to: files\gee_images\4100103_Abatiá
Using shapefile: files\municipal_shapefiles\4100202_Adrianópolis\4100202.shp
Skipping 4100202 (Adrianópolis): folder exists with 108 files
Download complete! Files saved to: files\gee_images\4100202_Adrianópolis
Using shapefile: files\municipal_shapefiles\4100301_Agudos do Sul\4100301.shp
Skipping 4100301 (Agudos do Sul): folder exists with 78 files
Download complete! Files saved to: files\gee_images\4100301_Agudos do Sul
Using shapefile: files\municipal_shapefiles\4100400_Almirante Tamandaré\4100400.shp
Skipping 4100400 (Almirante Tamandaré): folder exists with 90 files
Download complete! Files saved to: files\gee_images\4100400_Almirante Tamandaré
Using shapefile: files\municipal_shapefiles\4100459_Altamira do Paraná\4100459.shp
Skipping 4100459 (Altamira do Paraná): folder exists with 150 file